In [1]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from dateutil.parser import parse
import pandas as pd
import datetime
from dateutil import parser
import re

In [14]:
#downloaded from BVBRC... stupidly have to download fasta file separately from .csv with metadata
#get date and put in 'yyyy-mm-dd' format
#get country


#read in metadata file
metadata = pd.read_csv('BVBRC_metadata.csv')

accessions_from_bvbrc = []
seq_records = []

#read in sequences
for record in SeqIO.parse(open(f"BVBRC_sequences.fasta","r"), "fasta"):
    if len(record.seq) > 20000:
        accession, strain_name, collection_date, collection_date, formatted_date, country = 'None', 'None', 'None', 'None', 'None', 'None'
        accession = str(record.description.split('   ')[0].split('|')[1])
        strain_name = str(record.description.split('   ')[2].split('|')[0][1:-1])
        meta_row = metadata[metadata['GenBank Accessions']==accession]
        collection_date = meta_row['Collection Date'].item()
        
        
        if type(collection_date)!=float:

            if collection_date == '2016-2017':
                formatted_date = '2016-XX-XX'
            elif len(collection_date)==4:
                formatted_date = collection_date+'-XX-XX'
            elif len(collection_date) == 7:
                formatted_date = collection_date+'-XX'
            else:
                formatted_date = parser.parse(collection_date).strftime('%Y-%m-%d')

            country = str(meta_row['Isolation Country'].item())

  

            list_of_info = [accession, strain_name, formatted_date, country]
            new_record_info = '|'.join(list_of_info)
            accessions_from_bvbrc.append(accession)
            seq_records.append(SeqRecord(record.seq, id=new_record_info, description=''))



SeqIO.write(seq_records, f'229e_frombvbrc.fasta', "fasta")



49

In [15]:
#combine data from bvbrc and genbank that isn't redundant
combined_seq_records = seq_records

accessions_from_genbank = []

for record in SeqIO.parse(open(f"229e_fromgenbank.fasta","r"), "fasta"):
    accession = record.id.split('|')[0]
    if accession not in accessions_from_bvbrc:
        accessions_from_genbank.append(accession)
        combined_seq_records.append(SeqRecord(record.seq, id=record.id, description=''))
        
SeqIO.write(combined_seq_records, f'229e_genome.fasta', "fasta")

69